In [0]:
from pyspark.sql.functions import col

### Transactions table

In [0]:
df_transactions = spark.read.table('retail_cat.bronze.brz_transactions')
display(df_transactions)

In [0]:
df_products = spark.read.table('retail_cat.bronze.brz_products')
display(df_products)

In [0]:
df_stores = spark.read.table('retail_cat.bronze.brz_stores')
display(df_stores)

In [0]:
df_customers = spark.read.table('retail_cat.bronze.brz_customers')


In [0]:
df_customers = df_customers.withColumn('registration_date', col('registration_date').cast('date'))
display(df_customers)

In [0]:
# Check duplicates in the customers DF using email
df_customers.groupBy('email').count().filter(col('count') > 1).display()

In [0]:
# Check duplicates in the customers DF using Id
df_customers.groupBy('customer_id').count().filter(col('count') > 1).display()

In [0]:
# Check duplicates in the customers DF using Phone
df_customers.groupBy('phone').count().filter(col('count') > 1).display()

### Join all the data sets into a single dataset for Silver layer

In [0]:
df_silver = df_transactions \
    .join(df_customers, "customer_id") \
    .join(df_products, "product_id") \
    .join(df_stores, "store_id") \
    .withColumn("total_amount", col("quantity") * col("price"))

In [0]:
display(df_silver)

In [0]:
# create a silver table with this DF (this is a MANAGED table as we do this inside DB)
# You can create a EXTERNAL table if you want to dump this data into the ADLS
df_silver.write.format("delta") \
  .mode("overwrite") \
  .option("mergeSchema", "true") \
  .saveAsTable(f"retail_cat.silver.slv_retail")


In [0]:
%sql
-- check the table exists
select count(1) from retail_cat.silver.slv_retail;